In [5]:
!export CUDA_VISIBLE_DEVICES=0

In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
# os.environ['CUDA_VISIBLE_DEVICES']='0,1'
os.environ['CUDA_LAUNCH_BLOCKING']='1'
import torch
# torch.cuda.set_device(0)
print(torch.cuda.current_device())


0


In [5]:
torch.cuda.is_available()

True

In [6]:
torch.cuda.device_count()

1

In [6]:
torch.cuda.current_device()

0

In [8]:
# cudaSetDevice(0)
#dataset transcript要在dataset資好夾調
#cuda 在basesolver init調
#dataset要在data.py設定

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
# os.environ['CUDA_VISIBLE_DEVICES']='0,1'
os.environ['CUDA_LAUNCH_BLOCKING']='1'
import torch
# torch.cuda.set_device(0)
print(torch.cuda.current_device())
#! python
# -*- coding: utf-8 -*-
# Author: kun
# @Time: 2019-10-29 20:29

import yaml
import torch
import argparse
import numpy as np
import pandas as pd

dfs = []
alldf =pd.DataFrame()

class Para(object):
    a=1

def force_cudnn_initialization():
    s = 32
    dev = torch.device('cuda:0')
    torch.nn.functional.conv2d(torch.zeros(s, s, s, s, device=dev), torch.zeros(s, s, s, s, device=dev))
    
#force_cudnn_initialization()
def main():
    # For reproducibility, comment these may speed up training
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # Arguments
#     parser = argparse.ArgumentParser(description='Training E2E asr.')
#     parser.add_argument('--config', type=str, help='Path to experiment config.')
#     parser.add_argument('--name', default=None, type=str, help='Name for logging.')
#     parser.add_argument('--logdir', default='log/', type=str,
#                         help='Logging path.', required=False)
#     parser.add_argument('--ckpdir', default='ckpt/', type=str,
#                         help='Checkpoint path.', required=False)
#     parser.add_argument('--outdir', default='result/', type=str,
#                         help='Decode output path.', required=False)
#     parser.add_argument('--load', default=None, type=str,
#                         help='Load pre-trained model (for training only)', required=False)
#     parser.add_argument('--seed', default=0, type=int,
#                         help='Random seed for reproducable results.', required=False)
#     parser.add_argument('--cudnn-ctc', action='store_true',
#                         help='Switches CTC backend from torch to cudnn')
#     parser.add_argument('--njobs', default=32, type=int,
#                         help='Number of threads for dataloader/decoding.', required=False)
#     parser.add_argument('--cpu', action='store_true', help='Disable GPU training.')
#     parser.add_argument('--no-pin', action='store_true',
#                         help='Disable pin-memory for dataloader')
#     parser.add_argument('--test', action='store_true', help='Test the model.')
#     parser.add_argument('--no-msg', action='store_true', help='Hide all messages.')
#     parser.add_argument('--lm', action='store_true',
#                         help='Option for training RNNLM.')
#     # Following features in development.
#     parser.add_argument('--amp', action='store_true', help='Option to enable AMP.')
#     parser.add_argument('--reserve-gpu', default=0, type=float,
#                         help='Option to reserve GPU ram for training.')
#     parser.add_argument('--jit', action='store_true',
#                         help='Option for enabling jit in pytorch. (feature in development)')
#     ###
#     paras = parser.parse_args()
    paras = Para()
#     paras.config = './config/aishell_asr_example_lstm4atthead1-test.yaml'
#     paras.name = None
#     paras.logdir = 'log/'
#     paras.ckpdir = 'ckpt/'
#     paras.outdir = 'result/'
#     paras.load = None
#     paras.seed = 0
#     paras.cudnn_ctc = False
#     paras.cpu = False
#     paras.no_pin = False
#     paras.test = True
#     paras.no_msg = False
#     paras.lm = False
#     paras.amp = False
#     paras.reserve_gpu = 0
#     paras.jit = False
    setattr(paras, 'config', 'config/cv11Lu_asr_lstm4atthead_allvocab-MMSE-test-5-1.yaml')
    setattr(paras, 'name', None)
    setattr(paras, 'logdir', 'log/')
    setattr(paras, 'ckpdir', 'LAS_ckpt')
    setattr(paras, 'outdir', 'result/')
    setattr(paras, 'load', None)
    setattr(paras, 'seed', 0)
    setattr(paras, 'cudnn_ctc', False)
    setattr(paras, 'njobs',0)
    setattr(paras, 'cpu', False)
    setattr(paras, 'no_pin', False)
    setattr(paras, 'test', True)
    setattr(paras, 'no_msg', False)
    setattr(paras, 'lm', False)
    setattr(paras, 'amp', False)
    setattr(paras, 'reserve_gpu', 9)
    setattr(paras, 'jit', False)
    setattr(paras, 'gpu', not paras.cpu)
    setattr(paras, 'pin_memory', not paras.no_pin)
    setattr(paras, 'verbose', not paras.no_msg)
    setattr(paras, 'finetune', False)
    setattr(paras, 'binaryClass', True)
#     force_cudnn_initialization()

    config = yaml.load(open(paras.config, 'r'), Loader=yaml.FullLoader)

    np.random.seed(paras.seed)
    torch.manual_seed(paras.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(paras.seed)

    # Hack to preserve GPU ram just incase OOM later on server
    if paras.gpu and paras.reserve_gpu > 0:
        buff = torch.randn(int(paras.reserve_gpu * 1e9 // 4)).cuda()
        del buff

    if paras.lm:
        # Train RNNLM
        from train_lm import Solver

        mode = 'train'
    else:
        if paras.test:
            # Test ASR
            assert paras.load is None, 'Load option is mutually exclusive to --test'
            from Whisper_MMSE_bert_test import Solver

            mode = 'test'
        elif paras.finetune:
            assert paras.load is not None
            from finetune_asr import Solver
            mode = 'train'
        elif paras.binaryClass:
            assert paras.load is not None
            from Whisper_MMSE_bert_train import Solver
            mode = 'train'
        else:
            # Train ASR
            from train_asr import Solver

            mode = 'train'

    print("\nUsing {} mode\n".format(mode))
    for times in range (15,16):
        alldf =pd.DataFrame()
        for idx in range(5):
            paras.config = f'config/cv11Lu_asr_lstm4atthead_allvocab-MMSE-test-5-{idx+1}.yaml'
            config = yaml.load(open(paras.config, 'r'), Loader=yaml.FullLoader)
            config['core']['ckpt'] = f'LAS_ckpt/cv11Lu_asr_lstm4atthead_allvocab-MMSE-5fold-{idx+1}_sd0/bert-v2-15k-r-final-2.pth'
            solver = Solver(config, paras, mode)

            solver.load_data()
        #     solver.print_model()
            solver.set_model()
            names, hyps, txts, ans = solver.exec()
            dfs.append(pd.DataFrame(
                {'name': names,
                 'hyps': hyps,
                 'truth': txts,
                 'ifcorrect': ans
                }))
            print(dfs[idx])
            print(names)
            print(hyps)
            print(txts)
            print(ans)
            del solver
        for idx, df in enumerate(dfs):
            alldf = pd.concat([alldf, df])#
        #     df.to_csv(f'MMSE_result/bert-v2-{times}k{idx+1}.csv', index=False)
        # alldf.to_csv(f'MMSE_result/bert-v2-{times}k.csv', index=False) 
#             df.to_csv(f'./MMSE_result/5folds{times}k_oldASR_4LSTM{idx+1}.csv', index=False)
#         alldf.to_csv(f'./MMSE_result/5folds{times}k_oldASR_4LSTM.csv', index=False) 


0


In [2]:
main()


Using test mode

[INFO] Evaluating result of tr. config @ ../LAS_Mandarin_PyTorch-master/config/cv11Lu_asr_lstm4atthead_allvocab-MMSE-5fold-1.yaml
[INFO] Evaluating result of tr. ckpt @ LAS_ckpt/cv11Lu_asr_lstm4atthead_allvocab-MMSE-5fold-1_sd0/bert-v2-15k-r-final-2.pth


Some weights of the model checkpoint at google-bert/bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDatasetMMSE as Dataset
[VAGDataset] path: data_process, split: ['CTT5-1-16']
Mozillacv11Dataset CTT5-1-16 found wav data: 17
text len: 17
remove None, then wav data: 17, text len: 17
[VAGDataset] path: data_process, split: ['CTTsegment_remove']
Mozillacv11Dataset CTTsegment_remove found wav data: 99
text len: 99
remove None, then wav data: 87, text len: 87
Setup ASR model and optimizer 
('15',)   26.7041015625   30.0   False
('16',)   28.50641441345215   16.0   False
('22',)   28.1402587890625   29.0   False
('28',)   26.510557174682617   26.0   False
('34',)   28.95730972290039   30.0   False
('35',)   28.17825698852539   26.0   False
('40',)   22.541397094726562   26.0   False
('42',)   30.332603454589844   30.0   False
('51',)   27.25969696044922   30.0   False
('53',)   28.26171112060547   30.0   False
('55',)   2

Some weights of the model checkpoint at google-bert/bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDatasetMMSE as Dataset
[VAGDataset] path: data_process, split: ['CTT5-2-16']
Mozillacv11Dataset CTT5-2-16 found wav data: 18
text len: 18
remove None, then wav data: 18, text len: 18
[VAGDataset] path: data_process, split: ['CTTsegment_remove']
Mozillacv11Dataset CTTsegment_remove found wav data: 99
text len: 99
remove None, then wav data: 87, text len: 87
Setup ASR model and optimizer 
('11',)   25.608739852905273   16.0   False
('25',)   28.267902374267578   15.0   False
('2',)   29.931427001953125   30.0   False
('37',)   26.268898010253906   16.0   False
('39',)   19.579872131347656   14.0   False
('45',)   27.434812545776367   14.0   False
('46',)   24.03287124633789   30.0   False
('65',)   26.788009643554688   25.0   False
('67',)   28.500701904296875   24.0   False
('68',)   24.27304458618164   30.0   False
('

...

Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDatasetMMSE as Dataset
[VAGDataset] path: data_process, split: ['CTT5-3-16']
Mozillacv11Dataset CTT5-3-16 found wav data: 17
text len: 17
remove None, then wav data: 17, text len: 17
[VAGDataset] path: data_process, split: ['CTTsegment_remove']
Mozillacv11Dataset CTTsegment_remove found wav data: 99
text len: 99
remove None, then wav data: 87, text len: 87
Setup ASR model and optimizer 
('18',)   30.023616790771484   16.0   False
('1',)   29.746660232543945   30.0   False
('20',)   26.18621063232422   23.0   False
('30',)   29.05779457092285   30.0   False
('32',)   29.02469253540039   24.0   False
('38',)   27.0952205657959   28.0   False
('3',)   29.55388641357422   30.0   False
('43',)   29.543807983398438   30.0   False
('52',)   27.94156837463379   30.0   False
('57',)   28.443660736083984   23.0   False
('58',) 

Some weights of the model checkpoint at google-bert/bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDatasetMMSE as Dataset
[VAGDataset] path: data_process, split: ['CTT5-4-16']
Mozillacv11Dataset CTT5-4-16 found wav data: 17
text len: 17
remove None, then wav data: 17, text len: 17
[VAGDataset] path: data_process, split: ['CTTsegment_remove']
Mozillacv11Dataset CTTsegment_remove found wav data: 99
text len: 99
remove None, then wav data: 87, text len: 87
Setup ASR model and optimizer 
('100',)   29.128318786621094   30.0   False
('12',)   27.109451293945312   20.0   False
('17',)   24.191152572631836   26.0   False
('21',)   28.377859115600586   28.0   False
('29',)   30.550870895385742   30.0   False
('4',)   26.11554527282715   23.0   False
('54',)   30.040658950805664   30.0   False
('56',)   30.003385543823242   30.0   False
('63',)   19.60759162902832   13.0   False
('6',)   28.027057647705078   30.0   False
('

Some weights of the model checkpoint at google-bert/bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDatasetMMSE as Dataset
[VAGDataset] path: data_process, split: ['CTT5-5-16']
Mozillacv11Dataset CTT5-5-16 found wav data: 17
text len: 17
remove None, then wav data: 16, text len: 16
[VAGDataset] path: data_process, split: ['CTTsegment_remove']
Mozillacv11Dataset CTTsegment_remove found wav data: 99
text len: 99
remove None, then wav data: 87, text len: 87
Setup ASR model and optimizer 
('10',)   27.943212509155273   24.0   False
('13',)   28.18389129638672   30.0   False
('14',)   30.093429565429688   30.0   False
('24',)   26.15557098388672   13.0   False
('31',)   23.841001510620117   25.0   False
('44',)   30.134403228759766   30.0   False
('47',)   26.858388900756836   30.0   False
('50',)   29.170040130615234   30.0   False
('5',)   28.909713745117188   27.0   False
('64',)   24.32187271118164   19.0   False
('6

In [3]:
for idx, df in enumerate(dfs):
    alldf = pd.concat([alldf, df])#
    # df.to_csv(f'./MMSE_result/5folds12knew_4LSTM{idx+1}.csv', index=False)
alldf.to_csv(f'./MMSE_result/bert-v2-15k-r-final-2.csv', index=False)    

In [2]:
main()


Using test mode

[INFO] Evaluating result of tr. config @ ../LAS_Mandarin_PyTorch-master/config/cv11Lu_asr_lstm4atthead_allvocab-biclass2-5fold-1.yaml
Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDataset as Dataset
[VAGDataset] path: /home/jupyter-jason5/data_process, split: ['CTTtest']
Mozillacv11Dataset CTTtest found wav data: 21
text len: 21
remove None, then wav data: 21, text len: 21
[VAGDataset] path: /home/jupyter-jason5/data_process, split: ['CTTtrain']
Mozillacv11Dataset CTTtrain found wav data: 73
text len: 73
remove None, then wav data: 73, text len: 73
[INFO] Data spec. | Corpus = vag (from /home/jupyter-jason5/data_process)                                  
[INFO]            | Train sets = ['CTTtrain']	| Number of utts = 73                                        
[INFO]            | Dev sets = ['CTTtest']	| Number of utts = 21                      

In [3]:
from sklearn.metrics import classification_report, confusion_matrix
y_true = [0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0] 
y_pred = [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0]
target_names = ['non-patient', 'patient']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

 non-patient       0.92      0.80      0.86        15
     patient       0.62      0.83      0.71         6

    accuracy                           0.81        21
   macro avg       0.77      0.82      0.79        21
weighted avg       0.84      0.81      0.82        21



In [6]:
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
print('sensitivity: ',tp / (fn+tp))
print('specificity: ',tn / (fp+tn))


sensitivuty:  0.8333333333333334
specificity:  0.8


In [10]:
import numpy as np
from sklearn import metrics
y = np.array([0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0])
pred = np.array([0.0005, 0.1006, 0.8517, 0.9374, 0.9875, 0.2795, 0.0041, 0.0005, 0.0005, 0.2542, 0.0002, 0.0014, 0.0029, 0.9852, 0.0029, 3.6719e-05, 0.9708, 0.0003, 1.2828e-05, 0.9127, 0.0006])
fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)
metrics.auc(fpr, tpr)

0.7692307692307693

In [11]:
!python eval.py --file ~/LAS_Mandarin_PyTorch-master/result/mozillacv11_asr_stm4atthead-test_test_output.csv


============  Result of /home/jupyter-jason3/LAS_Mandarin_PyTorch-master/result/mozillacv11_asr_stm4atthead-test_test_output.csv ============
 -----------------------------------------------------------------------
| Statics		|  Truth	|  Prediction	| Abs. Diff.	|
 -----------------------------------------------------------------------
| Avg. # of chars	|  8.01	|  8.10	|  0.16		|
| Avg. # of words	|  1.00	|  1.00	|  0.00		|
 -----------------------------------------------------------------------
 ---------------------------------------------------------------
| Error Rate (%)| Mean		| Std.		| Min./Max.	|
 ---------------------------------------------------------------
| Character	| 7.9479	| 21.68		| 0.00/240.00	|
| Word		| 18.1725	| 38.56		| 0.00/100.00	|
 ---------------------------------------------------------------
Note : If the text unit is phoneme, WER = PER and CER is meaningless.

